In [1]:
!pip install apache-airflow

     ---------------------------------------- 0.0/44.6 kB ? eta -:--:--
     --------------------------- ------------ 30.7/44.6 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 44.6/44.6 kB 440.3 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached setuptools-75.6.0-py3-none-any.whl.metadata (6.7 kB)
     ---------------------------------------- 0.0/69.9 kB ? eta -:--:--
     ---------------------------------------- 69.9/69.9 kB 1.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/13.4 MB ? eta -:--:--
   ---------------------------------------- 0.1/13.4 MB 6.4 MB/s eta 0:00:03
    --------------------------------------- 0.2/13.4 MB 2.4 MB/s eta 0:

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.2 requires numpy<2.1.0,>=2.0.0; python_version >= "3.9", but you have numpy 2.1.3 which is incompatible.

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 434.5/434.5 kB 3.4 MB/s eta 0:00:00
  Created wheel for python-nvd3: filename=python_nvd3-0.16.0-py3-none-any.whl size=37613 sha256=6ad6ae4f11d47f3cf5030694bc608495a3e9326b5dd153ee742eacb477456287
  Stored in directory: c:\users\hp\appdata\local\pip\cache\wheels\1c\8b\d8\965c9856f31091a195a0e960b0edba176f326749b56cd7b875
Successfully built python-nvd3
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.35
    Uninstalling SQLAlchemy-2.0.35:
      Successfully uninstalled SQLAlchemy-2.0.35
  Attempting uninstall: setuptools
    Found existing installation: setuptools 58.1.0
    Uninstalling setuptools-58.1.0:
      Successfully uninstalled setuptools-58.1.0


In [ ]:
import pendulum
from airflow.decorators import dag, task
from airflow.hooks.base import BaseHook
import pandas as pd
import os
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError

# Конфигурация подключения PostgreSQL через Airflow Connection
POSTGRES_CONN_ID = 'superset_db'  # Убедитесь, что подключение настроено в Airflow

# Заглушка локального пути
LOCAL_REPO_PATH = r'C:\Users\HP\Documents\GitHub\ST0RVG3\airflow\app\data'

CSV_FILES = {
    'Customers': 'Customers.csv',
    'CreditProducts': 'CreditProducts.csv',
    'CreditAgreements': 'CreditAgreements.csv',
    'TransactionTypes': 'TransactionTypes.csv',
    'CreditTransactions': 'CreditTransactions.csv'
}

CREATE_TABLES_SQL = """
CREATE TABLE IF NOT EXISTS Customers (
    CustomerID SERIAL PRIMARY KEY,
    CustomerTypeID INTEGER NOT NULL CHECK (CustomerTypeID IN (1, 2)),
    Name VARCHAR(255) NOT NULL,
    DateOfBirth DATE,
    RegistrationDate DATE,
    TIN VARCHAR(50) NOT NULL UNIQUE,
    ContactInfo VARCHAR(255),
    CHECK (
        (CustomerTypeID = 1 AND DateOfBirth IS NOT NULL AND RegistrationDate IS NULL) OR
        (CustomerTypeID = 2 AND RegistrationDate IS NOT NULL AND DateOfBirth IS NULL)
    )
);
"""

# CREATE TABLE IF NOT EXISTS CreditProducts (
#     CreditProductID SERIAL PRIMARY KEY,
#     ProductName VARCHAR(255) NOT NULL,
#     InterestRate DECIMAL(5, 2) NOT NULL CHECK (InterestRate >= 0),
#     MaxLoanAmount NUMERIC(15, 2) NOT NULL CHECK (MaxLoanAmount > 0),
#     MinRepaymentTerm INTEGER NOT NULL CHECK (MinRepaymentTerm > 0),
#     CollateralRequired BOOLEAN NOT NULL DEFAULT FALSE
# );

# CREATE TABLE IF NOT EXISTS CreditAgreements (
#     CreditAgreementID SERIAL PRIMARY KEY,
#     CustomerID INTEGER NOT NULL,
#     CreditProductID INTEGER NOT NULL,
#     AgreementDate DATE NOT NULL,
#     LoanAmount NUMERIC(15, 2) NOT NULL CHECK (LoanAmount > 0),
#     LoanTerm INTEGER NOT NULL CHECK (LoanTerm > 0),
#     InterestRate DECIMAL(5, 2) NOT NULL CHECK (InterestRate >= 0),
#     FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID) ON DELETE CASCADE,
#     FOREIGN KEY (CreditProductID) REFERENCES CreditProducts(CreditProductID) ON DELETE RESTRICT
# );

# CREATE TABLE IF NOT EXISTS TransactionTypes (
#     TransactionTypeID SERIAL PRIMARY KEY,
#     TransactionTypeName VARCHAR(100) NOT NULL UNIQUE
# );

# CREATE TABLE IF NOT EXISTS CreditTransactions (
#     TransactionID SERIAL PRIMARY KEY,
#     CustomerID INTEGER NOT NULL,
#     CreditAgreementID INTEGER NOT NULL,
#     TransactionDate DATE NOT NULL,
#     TransactionAmount NUMERIC(15, 2) NOT NULL,
#     TransactionTypeID INTEGER NOT NULL,
#     FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID) ON DELETE CASCADE,
#     FOREIGN KEY (CreditAgreementID) REFERENCES CreditAgreements(CreditAgreementID) ON DELETE CASCADE,
#     FOREIGN KEY (TransactionTypeID) REFERENCES TransactionTypes(TransactionTypeID) ON DELETE RESTRICT
# );
# """

default_args = {
    "owner": "airflow",
    'retries': 1,
    'retry_delay': pendulum.duration(minutes=5),
}

@dag(
    default_args=default_args,
    dag_id='create_tables_and_load_csv',
    schedule_interval=None,  # Запуск вручную
    start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),
    catchup=False,
    tags=['postgres', 'csv', 'setup'],
    render_template_as_native_obj=True,
)
def create_tables_and_load_csv():


    @task()
    def create_tables():
        """
        Создаёт таблицы в базе данных PostgreSQL.
        """
        connection = BaseHook.get_connection(POSTGRES_CONN_ID)
        engine = create_engine(connection.get_uri())
        print(engine)

        # try:
        #     with engine.connect() as conn:
        #         conn.execute(text(CREATE_TABLES_SQL))
        #         print("Таблицы успешно созданы или уже существуют.")
        # except SQLAlchemyError as e:
        #     print(f"Ошибка при создании таблиц: {e}")
        #     raise


    @task()
    def read_csv_files():
        """
        Читает CSV-файлы из локального репозитория и возвращает их в виде словаря pandas DataFrame.
        """
        data_frames = {}
        for table, filename in CSV_FILES.items():
            file_path = os.path.join(LOCAL_REPO_PATH, filename)
            if os.path.exists(file_path):
                try:
                    df = pd.read_csv(file_path)
                    data_frames[table] = df
                    print(f"Файл {filename} успешно прочитан для таблицы {table}.")
                except pd.errors.ParserError as e:
                    raise Exception(f"Ошибка при чтении файла {filename}: {e}")
            else:
                raise FileNotFoundError(f"Файл {filename} не найден по пути {file_path}.")
        return data_frames


    @task()
    def insert_data_into_postgres(data_frames: dict):
        """
        Вставляет данные из pandas DataFrame в соответствующие таблицы.
        """
        connection = BaseHook.get_connection(POSTGRES_CONN_ID)
        engine = create_engine(connection.get_uri())

        for table, df in data_frames.items():
            try:
                df.to_sql(table, engine, if_exists='append', index=False)
                print(f"Данные из {table} успешно вставлены.")
            except SQLAlchemyError as e:
                print(f"Ошибка при вставке данных в таблицу {table}: {e}")
                raise

    create_tables_task = create_tables()
    read_csv_task = read_csv_files()
    insert_data_task = insert_data_into_postgres(read_csv_task)

    create_tables_task >> read_csv_task >> insert_data_task

dag = create_tables_and_load_csv()
